# 深入学习 DEC 模型

第三节，我通过吃百家饭复现了 [DEC 模型](https://arxiv.org/abs/1511.06335)。不知道是不是我的问题，[vlukiyanov/pt-dec](https://github.com/vlukiyanov/pt-dec) 仓库的 DEC 模型跑起来 loss 不降反升（见 [test_ptdec.ipynb](./archived/test_ptdec.ipynb)）。好在该仓库已经实现了 DEC 论文中几个重要的类和函数，将它们拼接一番，也顺利把模型跑起来了。

上一节，我们比较了 DEC 模型和传统的 K-Means 模型的准确率。在我的数据集上，它们准确率类似，都在 0.7 左右。这个结果不意外，因为 DEC 本身就是用 K-Means 来初始化聚类中心的。

经过前两节的工作，我们对 DEC 模型有了初步的理解，希望在这里停下来总结一下。

## 1. 模型的创新点

这个模型的主要创新点是用软分配分布逼近一个目标分布，使得聚类下的样本特征向聚类中心靠拢，同时远离其他聚类中心。但是在初次使用 K-Means 分配聚类的时候，准确率就非常高了。可能是我数据集或者训练技巧的原因，这个创新点对我结果的提升不太明显。

下面来介绍一下，软分配向目标分配逼近的具体过程：

1. **聚类中心初始化**：首先做聚类中心的初始化，得到每个聚类的特征向量
2. **软分配策略**：然后计算归一化的软分配分布 $q$
3. **目标分布优化**：再通过 $q$ 计算分布“更尖锐”的目标分布 $p$
4. **模型训练**：最后通过 KL 散度，最小化软分配分布与目标分布的差异，反向更新参数

以下是对具体过程更详细的解释。

### 1.1 聚类中心初始化

通过编码器将原始数据映射到低维特征空间后，用 K-Means 计算 $k$ 个簇的类心。

$k$ 个类心组成 $(k, hidden\_dim)$ 维矩阵，其中 `hidden_dim` 是编码器最后一层的维度大小。

### 1.2 软分配策略

样本经过编码器映射成一个 `hidden_dim` 维特征向量，因此可以与 $k$  个同样是 `hidden_dim` 维的聚类中心计算欧式距离。

我们需要借由学生 t 分布，将样本与各个类心之间的距离，转换成样本分配到各个类心的概率。

原始的学生 t 分布如下：

$f(t) = \frac{\Gamma\left(\frac{\alpha + 1}{2}\right)}{\sqrt{\alpha\pi} \cdot \Gamma\left(\frac{\alpha}{2}\right)} \cdot \left(1 + \frac{t^2}{\alpha}\right)^{-\frac{\alpha + 1}{2}}$

其中：

- $\Gamma$ 是伽马函数
- $\alpha$ 是自由度

t 分布用于计算样本嵌入与聚类中心的相似度，并将其转化为概率分布。具体步骤如下：

**1）距离计算**

假设样本嵌入为 $z_i$，聚类中心为 $\mu_i$，计算欧式距离的平方：

$d_{ij}^2 = \|z_i - \mu_j\|^2$

**2）t 分布概率转换**

将距离代入 t 分布的概率密度函数形式，得到非归一化的概率：

$q_{ij} \propto \left(1 + \frac{d_{ij}^2}{\alpha}\right)^{-\frac{\alpha + 1}{2}}$

其中 $\alpha$ 是自由度参数。

该公式通过调整距离的权重，使得近邻样本的概率更高，远邻样本的概率更低。

**3）归一化**

下面是样本 $i$ 分配到聚类 $j$ 的归一化概率：

$q_{ij} = \frac{\left(1 + \frac{d_{ij}^2}{\alpha}\right)^{-\frac{\alpha + 1}{2}}}{\sum_{j'} \left(1 + \frac{d_{ij'}^2}{\alpha}\right)^{-\frac{\alpha + 1}{2}}}$

其中，样本 $i$ 对所有聚类中心的 t 分布概率为分母，对第 $j$ 个聚类中心的 t 分布概率为分子。归一化确保样本 $i$ 对所有聚类中心的概率和为 1.

> 软分配 (Soft Assignment) 为每个样本分配一个属于各个聚类中心的概率分布，而不是硬性地将其分配到某个聚类中。

### 1.3 目标分布优化

目标分布 (Target Distribution) 用于引导模型的训练，使得聚类中心更分离，聚类内样本更紧凑。

目标分布的计算公式如下：

$p_{ij} = \frac{q_{ij}^2 / f_j}{\sum_{j'=1}^k q_{ij'}^2 / f_{j'}}$

其中，$f_j = \sum_{i} q_{ij}$ 是第 $j$ 个聚类的软分配频率。

该分布通过提升高频聚类的置信度 并抑制低频聚类的噪声 ，引导模型学习更具判别性的特征。

### 1.4 模型训练

使用 KL 散度作为损失函数，最小化软分配分布与目标分布之间的差异。具体来说，损失函数定义为：

$L = \text{KL}(P \| Q) = \sum_{i,j} p_{ij} \log \frac{p_{ij}}{q_{ij}}$

其中，$P$ 是目标分布，$Q$ 是软分配分布。通过反向传播算法更新编码器的参数和聚类中心，不断迭代训练模型，直到损失函数收敛或达到预设的训练轮数。



## 2. 训练阶段

DEC 模型训练大致分为三个阶段：

1. 预训练自编码器
2. 初始化聚类中心
3. 最小化软分配与目标分布的差异

我把训练过程写到本仓库的 [dec.py](./dec.py) 文件中了。

## 3. 聚类标签匹配问题

>  参考：[匈牙利算法](https://zh.wikipedia.org/zh-sg/%E5%8C%88%E7%89%99%E5%88%A9%E7%AE%97%E6%B3%95)

由于训练产生的标签与原标签之间可能标号不同，但是聚类下的样本却相同。为了对齐标号，需要求解 **线性分配问题**（Linear Assignment Problem, LAP），即寻找二分图的最小权匹配。其核心作用是将两个集合中的元素进行最优匹配，使得总匹配成本最小。我们可以直接用 scipy 的 [linear_sum_assignment](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html) 函数求解。

下面是 [vlukiyanov/pt-dec](https://github.com/vlukiyanov/pt-dec/blob/master/ptdec/utils.py) 仓库中计算聚类准确率的函数，写得挺好的，可以借鉴：

In [2]:
import numpy as np
from typing import Optional
from scipy.optimize import linear_sum_assignment

def cluster_accuracy(y_true, y_predicted, cluster_number: Optional[int] = None):
    """
    Calculate clustering accuracy after using the linear_sum_assignment function in SciPy to
    determine reassignments.

    :param y_true: list of true cluster numbers, an integer array 0-indexed
    :param y_predicted: list  of predicted cluster numbers, an integer array 0-indexed
    :param cluster_number: number of clusters, if None then calculated from input
    :return: reassignment dictionary, clustering accuracy
    """
    if cluster_number is None:
        cluster_number = (
            max(y_predicted.max(), y_true.max()) + 1
        )  # assume labels are 0-indexed
    count_matrix = np.zeros((cluster_number, cluster_number), dtype=np.int64)
    for i in range(y_predicted.size):
        count_matrix[y_predicted[i], y_true[i]] += 1

    row_ind, col_ind = linear_sum_assignment(count_matrix.max() - count_matrix)
    reassignment = dict(zip(row_ind, col_ind))
    accuracy = count_matrix[row_ind, col_ind].sum() / y_predicted.size
    return reassignment, accuracy

## 4. 模型推理和优化

1）模型推理

我在 `./reserved/dec_full.pth` 路径下保留了一个训练过的 DEC 模型用于推理。

In [4]:
import dec
import torch
import numpy as np

# 定义设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 声明模型结构
DEC = dec.DEC
ClusterAssignment = dec.ClusterAssignment

# 加载模型
full_model_path = './reserved/dec_full.pth'
loaded_model = dec.load_full_model(full_model_path, device=device)

# 执行推理
new_embeddings = np.random.randn(50, 768).astype(np.float32)
y_pred = dec.infer_embeddings(loaded_model, new_embeddings, device=device)
y_pred

Using device: cuda


array([63, 12, 76, 76, 18, 76, 63, 93, 74, 76, 76, 18, 24, 18, 74, 18, 24,
       63, 74, 18, 61,  6, 48, 18, 16, 24, 63, 74, 63, 76, 76, 85, 76, 41,
       24, 76, 63, 18, 39, 76, 24, 24, 14, 71, 76, 12, 76, 63, 24, 99],
      dtype=int64)

2）模型优化

使用 `dec.py` 训练模型，且进行一个模型优化尝试：在拟合目标分布阶段，冻结编码器参数。

PS：这个改造似乎没啥用 ...

In [5]:
import copy
import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

import dec

from torch.utils.data import DataLoader, TensorDataset

In [6]:
# 参数配置
TRAIN_CSV_PATH = './data/train_embed_label.csv'

# 模型超参数配置
config = {
    "dims": [768, 256, 32],
    "n_clusters": 100,
    "pretrain_epochs": 50,
    "soft_dist_epochs": 100,
    "update_interval": 10,
    "batch_size": 256,
    "tol": 0.001,
    "alpha": 1.0,
    "save_dir": "./model",
    "args_model_file": "dec_args.pth",
    "full_model_file": "dec_full.pth"
}

In [7]:
# 模型优化尝试：冻结编码器参数
def train_dec(model, data_loader, epochs, device, X, y_true=None, interval=10):
    """通过目标分布引导聚类优化"""

    # 记录最优模型
    best_model, best_acc = None, None

    for param in model.encoder.parameters():
        param.requires_grad = False
    optimizer = optim.Adam([
        {'params': model.assignment.parameters(), 'lr': 1e-5}
    ])

    criterion = F.kl_div
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for idx, x in data_loader:
            x = x.to(device)
            optimizer.zero_grad()
            output = model(x)
            target = dec.target_distribution(output).detach()
            loss = criterion(output.log(), target, reduction='batchmean')
            loss.backward()

            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(),
                                           max_norm=1.0,
                                           norm_type=2)

            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(data_loader)
        if (epoch + 1) % interval == 0:
            print(f"DEC Train Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        if y_true is not None:
            # 计算准确率
            with torch.no_grad():
                input = torch.from_numpy(X).float().to(device)
                y_pred = model(input).argmax(1).cpu().numpy()
            current_acc = dec.acc(y_true, y_pred)

            # 更新最优模型
            if best_acc is None or current_acc > best_acc:
                best_model = copy.deepcopy(model)
                best_acc = current_acc
                print(f'===== best_acc: {best_acc:.4f} =====')

    return model if best_model is None else best_model, best_acc

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 数据准备
X, y_true = dec.load_embed_data(TRAIN_CSV_PATH)
dataset = TensorDataset(torch.arange(len(X)), torch.from_numpy(X.astype(np.float32)))
pretrain_loader = DataLoader(dataset, batch_size=config["batch_size"], shuffle=True)

# ======= 阶段一：训练降噪自编码器 =======
# 实例化编码器
auto_encoder = dec.Autoencoder(config["dims"]).to(device)

# 执行编码器预训练代码
auto_encoder = dec.pretrain(autoencoder=auto_encoder,
                            data_loader=pretrain_loader,
                            epochs=50,
                            device=device,
                            interval=config["update_interval"])

# ======= 阶段二：初始化聚类中心 =======
full_loader = DataLoader(dataset, batch_size=1024, shuffle=False)
kmeans, y_pred, init_acc = dec.init_cluster_centers(encoder=auto_encoder.encoder,
                                                    data_loader=full_loader,
                                                    n_clusters=config["n_clusters"],
                                                    device=device,
                                                    y_true=y_true)
print(f'init_acc: {init_acc}')

# 代表聚类中心的特征向量
cluster_centers = torch.tensor(kmeans.cluster_centers_,
                               dtype=torch.float,
                               requires_grad=True,
                               device=device)

# ======= 阶段三：训练 DEC =======
# 实例化 DEC
dec_model = dec.DEC(
    cluster_number=config["n_clusters"],  # 预设的聚类数
    hidden_dimension=config["dims"][-1],  # 编码器输出维度
    encoder=auto_encoder.encoder,
    alpha=config["alpha"],
    cluster_centers=cluster_centers
)
data_loader = DataLoader(dataset, batch_size=config["batch_size"], shuffle=False)
dec_model, dec_acc = train_dec(model=dec_model,
                               data_loader=data_loader,
                               epochs=config["soft_dist_epochs"],
                               device=device,
                               X=X,
                               y_true=y_true,
                               interval=config["update_interval"])

# 保存最优模型
dec.save_full_model(dec_model, config)
dec.save_args_model(dec_model, config)

# 计算指标
y_pred = dec.infer_embeddings(dec_model, X, device=device)
print("\nFinal Clustering Results:")
print(f"ACC: {dec.acc(y_true, y_pred):.4f}")
print(f"NMI: {dec.nmi(y_true, y_pred):.4f}")
print(f"ARI: {dec.ari(y_true, y_pred):.4f}")

Using device: cuda
Pretrain Epoch 10/50, Loss: 0.0009
Pretrain Epoch 20/50, Loss: 0.0008
Pretrain Epoch 30/50, Loss: 0.0007
Pretrain Epoch 40/50, Loss: 0.0007
Pretrain Epoch 50/50, Loss: 0.0007
init_acc: 0.7061
===== best_acc: 0.7060 =====
===== best_acc: 0.7061 =====
DEC Train Epoch 10/100, Loss: 0.0050
DEC Train Epoch 20/100, Loss: 0.0053
DEC Train Epoch 30/100, Loss: 0.0057
DEC Train Epoch 40/100, Loss: 0.0061
DEC Train Epoch 50/100, Loss: 0.0065
DEC Train Epoch 60/100, Loss: 0.0069
DEC Train Epoch 70/100, Loss: 0.0073
DEC Train Epoch 80/100, Loss: 0.0077
DEC Train Epoch 90/100, Loss: 0.0081
DEC Train Epoch 100/100, Loss: 0.0085

Final Clustering Results:
ACC: 0.7061
NMI: 0.8057
ARI: 0.5622
